In [76]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np

In [4]:
nba_teams = teams.get_teams()
df_teams = pd.DataFrame(nba_teams)
df_teams.head(60)

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,San Francisco,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


# Selección de partido
Se va a seleccionar el partido jugado el 13/12/2025 entre `okc` y `spurs`.

In [38]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import leaguegamelog

In [48]:
partidos_temporada = leaguegamefinder.LeagueGameFinder(season_nullable="2025-26")
df_partidos_temporada = partidos_temporada.get_data_frames()[0]
df_partido_okc = df_partidos_temporada[((df_partidos_temporada['GAME_DATE'] == "2025-12-13") & 
                                    (df_partidos_temporada['TEAM_ABBREVIATION'] == 'OKC'))]
df_partido_okc

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22025,1610612760,OKC,Oklahoma City Thunder,0022501230,2025-12-13,SAS @ OKC,L,240,109,...,0.889,14,37,51,20,13,4,14,25,-2.0


In [50]:
game_id = df_partido_okc['GAME_ID'].loc[0]
print(game_id)

0022501230


In [ ]:
partido = df_partidos_temporada[df_partidos_temporada['GAME_ID'] == game_id]
partido

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22025,1610612760,OKC,Oklahoma City Thunder,0022501230,2025-12-13,SAS @ OKC,L,240,109,...,0.889,14,37,51,20,13,4,14,25,-2.0
2,22025,1610612759,SAS,San Antonio Spurs,0022501230,2025-12-13,SAS @ OKC,W,241,111,...,0.750,15,37,52,21,9,6,19,23,2.0


In [74]:
def extraer_sim(matchup):
    resultado = matchup[4]
    return resultado

def extraer_primero(matchup):
    resultado = matchup[0:3]
    return resultado

In [77]:
df_partidos_temporada['matchup_sim'] = df_partidos_temporada['MATCHUP'].loc[:].apply(extraer_sim)
df_partidos_temporada['matchup_pri'] = df_partidos_temporada['MATCHUP'].loc[:].apply(extraer_primero)
df_partidos_temporada['HA'] = np.where((df_partidos_temporada['TEAM_ABBREVIATION'] == df_partidos_temporada['matchup_pri'])&
    (df_partidos_temporada['matchup_sim'] == '@'), 'AWAY', np.where(
        (df_partidos_temporada['TEAM_ABBREVIATION'] != df_partidos_temporada['matchup_pri']) &
        (df_partidos_temporada['matchup_sim'] == 'v'),'AWAY', 'HOME'
    )
)
df_partidos_temporada

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,AST,STL,BLK,TOV,PF,PLUS_MINUS,marchup_sim,matchup_sim,matchup_pri,HA
0,22025,1610612760,OKC,Oklahoma City Thunder,0022501230,2025-12-13,SAS @ OKC,L,240,109,...,20,13,4,14,25,-2.0,@,@,SAS,HOME
1,22025,1610612752,NYK,New York Knicks,0022501229,2025-12-13,NYK @ ORL,W,240,132,...,28,9,7,12,23,12.0,@,@,NYK,AWAY
2,22025,1610612759,SAS,San Antonio Spurs,0022501230,2025-12-13,SAS @ OKC,W,241,111,...,21,9,6,19,23,2.0,@,@,SAS,AWAY
3,22025,1610612753,ORL,Orlando Magic,0022501229,2025-12-13,NYK @ ORL,L,241,120,...,28,9,3,14,24,-12.0,@,@,NYK,HOME
4,22025,1610612744,GSW,Golden State Warriors,0022501215,2025-12-12,GSW vs. MIN,L,240,120,...,27,7,2,13,23,-7.0,v,v,GSW,HOME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,12025,1610612756,PHX,Phoenix Suns,0012500001,2025-10-03,PHX @ LAL,W,241,103,...,29,9,5,16,31,22.0,@,@,PHX,AWAY
889,12025,1610612747,LAL,Los Angeles Lakers,0012500001,2025-10-03,LAL vs. PHX,L,240,81,...,10,9,7,22,25,-22.0,v,v,LAL,HOME
890,12025,15016,MEL,Melbourne United,0012500009,2025-10-03,MEL @ NOP,L,240,97,...,23,7,1,17,18,-10.2,@,@,MEL,AWAY
891,12025,1610612752,NYK,New York Knicks,0012500008,2025-10-02,PHI @ NYK,W,238,99,...,19,13,2,17,26,15.0,@,@,PHI,HOME
